# Sparse Sheaf Signal Processing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.neighbors import NearestNeighbors
from vdm import VDM
import cvxpy as cp
from sklearn.linear_model import OrthogonalMatchingPursuit
from wavelet import Wavelet
SEED = 6111983
np.random.seed(SEED)

##### Data

Two synthetic datasets are generated: a cube point cloud and a fibonacci sphere.

In [2]:
# Generate a cube in R^3 with uniformly random points
N = 1000 # number of points in the cloud
np.random.seed(6111983)
cube_point_cloud = np.random.uniform(-0.5,0.5,N*3).reshape((N, 3)) # points

# Function that computes a Fibonacci Sphere
def fibonacci_sphere(N):
    i = np.arange(N)
    phi = (1+np.sqrt(5))/2
    z = 1-2*i/(N-1)
    theta = 2*np.pi*i/phi
    r = np.sqrt(1-z*z)
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    return np.column_stack((x,y,z))

# Generate fibonacci_sphere
N = 1000 # number of points
sphere_points = fibonacci_sphere(N)

##### Vector Diffusion Mapping

The following code uses the VDM class to build a tangent vector space bundle and find the alignment matrices, saved in a dictionary O. The functions ```coboundary_map``` and ```sheaf_laplacian``` build the sheaf laplacian using as restriction maps the alignment matrices Oij, and as node and edge stalks $\mathbb{R}^2$.

In [3]:
eps = 0.15 # epsilon for the graph
eps_pca = 0.1 # epsilon for the PCA
k = 7 # number of nearest neighbors for the graph 
vdm2 = VDM(sphere_points,eps_pca,k) # VDM object
G = vdm2.make_graph(method='radius') # create the graph with the radius method
O = vdm2.make_alignment_matrices() # get alignment matrices

In [12]:
# Build sheaf Laplacian
# Define edge orientation as follows: edge (i,j) has tail i and head j with i<j
def coboundary_map(G,O,d):
    '''
    Function that builds the coboundary map of a sheaf given a graph G and O dictionary of alignment matrices Oij
    Assumption: all restriction maps (alignment matrices) have the same dimension dxd
    Inputs:
    G = graph
    O = dictionary of alignment matrices/ restriction maps (O[i][j] is the restriction map from node i to edge (i,j))
    d = dimension of the matrices
    Returns:
    delta = coboundary map
    '''
    num_edges = G.number_of_edges()
    num_nodes = G.number_of_nodes()
    delta = np.zeros((num_edges*d,num_nodes*d))
    for edge_idx, edge in enumerate(G.edges()):
        if edge[0]<edge[1]:
            i = int(edge[0])
            j = int(edge[1])
        else:
            i = int(edge[1])
            j = int(edge[0])
        delta[edge_idx*d:(edge_idx+1)*d, i*d:(i+1)*d] = O[i][j] # * G.get_edge_data(i,j)['weight']  # multiply by edge weight
        delta[edge_idx*d:(edge_idx+1)*d, j*d:(j+1)*d] = - O[j][i] # * G.get_edge_data(j,i)['weight']
    return delta

def sheaf_laplacian(G,O,d):
    delta = coboundary_map(G,O,d)
    L = delta.T @ delta
    return L

In [13]:
L = sheaf_laplacian(G,O,vdm2.estimate_dim())

##### Wavelets

The code below creates a Wavelet object and uses some of the class methods to find wavelet coefficients.

The kernel used by the wavelet is the following:

$$
g(x) = x^k \cdot \exp(-t \cdot x^p)
$$

with default values

* k=1
* t=1
* p=1

that can be modified using the method ```Wavelet.set_kernel_parameters(k,t,p)```. Only non-negative values are allowed.

This kernel was chosen such that $g(0)=0$ and $\lim_{x \rightarrow +\infty} g(x) = 0$, to serve as a bandpass filter.

In [14]:
# Create a wavelet object and compute the Laplacian eigendecomposition
wav = Wavelet(L)
wav.get_eig_laplacian()

(array([33.79120917, 33.79120917, 33.06391686, ...,  1.86879581,
         1.18599063,  1.18599063], shape=(2000,)),
 array([[-2.82993843e-03, -1.75835884e-06, -3.31831942e-07, ...,
         -7.95511670e-04, -5.30693815e-04, -7.24935232e-05],
        [-6.81276315e-16, -2.82993788e-03,  9.63673389e-04, ...,
         -1.10955098e-01, -9.37775755e-17, -5.25719140e-04],
        [ 3.03464499e-03,  8.73110703e-04, -1.17047724e-03, ...,
          9.99250732e-02, -1.70766241e-04,  4.47151446e-04],
        ...,
        [ 5.77087915e-07,  7.70750073e-08, -2.02938829e-05, ...,
          1.46573498e-03, -2.94556395e-02, -1.00995470e-01],
        [-1.26269743e-06, -1.56566149e-06, -5.80353599e-06, ...,
         -1.52192162e-03, -5.70139489e-03,  9.99864796e-02],
        [ 1.56487723e-06, -1.26172486e-06, -2.34912947e-05, ...,
          2.16477251e-04, -1.01718801e-01, -1.95428821e-02]],
       shape=(2000, 2000)))

In [15]:
# Find the wavelet coefficient of a signal f for a given scale and shift
scale = 0.5
shift = 0
np.random.seed(SEED)
f = np.random.randn(L.shape[0])
print(f"Wavelet coefficient of f for scale {scale} and shift {shift}: {wav.wavelet_coef(f,scale,shift)}")
# Show the value of the wavelet at node 2
value = wav.wavelet(0,scale,shift)
print(f"Value of the wavelet at node 0: {value}")

Wavelet coefficient of f for scale 0.5 and shift 0: 0.026248509956813472
Value of the wavelet at node 0: 0.00864084425196611


##### Dictionaries

The following code cells build three different dictionaries:

1. This dictionary uses 7 scales [0.25, 0.5, 1, 2, 4, 8, 16] and shifts [1,...,L.shape[0]], with L being the Laplacian. All shifts are used due to the discrete nature of the spatial domain. Scale are selected using geometrical spacing to analyze signals at different resolutions. Small scales pick up on local features, larger scales on global ones. For every scale and shift parameter, a wavelet will be added to the dictionary, which will have dimension L.shape[0] x (num_scales*num_shifts).

2. This dictionary was built by taking all shifts [1,...,L.shape[0]] and all scales [0.25, 0.5, 1, 2, 4, 8, 16], but instead of building wavelets for each pair of scale and shift parameters, it randomly selects combinations such that scale 0 is combined with all shifts, scale 1 with half of them, etc. The reasoning is that smaller scales are able to detect smaller details, so all shifts are required to cover the whole space. 

3. For this dictionary, geometric spacing was applied for the same reasons as in dictionary 1, but only for 4 scales, and using a more problem-specific range of scales, by king the reciprocals of the max and min eigenvalues of the Laplacian. We get again a dictionary of shape L.shape[0] x  (num_scales*num_shifts)

All dictionaries are L2-normalized, but the normalization can be set to False in the ```wav.make_dictionary()``` and ```wav.make_dictionary_from_tuples()``` functions. The difference between these two functions is that the first one computes wavelets for every (scale,shift) pair given lists of scales and shifts, while the second iterates over a list of tuples directly.

In [16]:
# DICTIONARY 1: scales and shifts
num_scales1 = 7
scales1 = [2**(j-2) for j in range(num_scales1)]
num_shifts1 = L.shape[0]
shifts1 = [i for i in range(num_shifts1)]
# Build the dictionary of wavelets
wav_dict1 = wav.make_dictionary(scales1,shifts1)

# DICTIONARY 2: scales and shifts
# Scale set
num_scales2 = 7
scales2 = [2**(j-2) for j in range(num_scales2)]
# Shift set
shifts_per_scale = {
    scales2[0]: L.shape[0],
    scales2[1]: L.shape[0]//2,
    scales2[2]: L.shape[0]//4,
    scales2[3]: L.shape[0]//8,
    scales2[4]: L.shape[0]//16,
    scales2[5]: L.shape[0]//32,
    scales2[6]: L.shape[0]//64,
}
shifts2 = {s: np.random.choice(L.shape[0], shifts_per_scale[s], replace=False) for s in scales2}
# Make scale_shift tuple list
scale_shift_tuples = [(s, shifts2[s][i]) for s in scales2 for i in range(shifts_per_scale[s])]
# Build the dictionary of wavelets
wav_dict2 = wav.make_dictionary_from_tuples(scale_shift_tuples)

# DICTIONARY 3: scales and shifts
# Scale set
lambda_max = np.max(wav.eigvals)
lambda_min = np.min(wav.eigvals[wav.eigvals>0])
s_min = 1/lambda_max
s_max = 1/lambda_min
num_scales3 = 4
scales3 = np.geomspace(s_min, s_max, num_scales3)
# Shift set
num_shifts3 = L.shape[0]
shifts3 = [i for i in range(num_shifts3)]
# Make a dictionary from wavelets with scales in scales3 and shifts in [1:N]
wav_dict3 = wav.make_dictionary(scales3,shifts3)

In [ ]:
# Function that computes the coherence of a dictionary
def mutual_coherence(D):
    D_norm = D / np.linalg.norm(D, axis=0, keepdims=True) # Column-wise normalization
    gram = D_norm.T @ D_norm # Gram matrix
    np.fill_diagonal(gram, 0)
    mu = np.max(np.abs(gram))
    return mu

# Print coherences
mu1 = mutual_coherence(wav_dict1)
print("Mutual coherence 1:", mu1)
mu2 = mutual_coherence(wav_dict2)
print("Mutual coherence 2:", mu2)
mu3 = mutual_coherence(wav_dict3)
print("Mutual coherence 3:", mu3)


Mutual coherence 1: 0.9999999998558362
Mutual coherence 2: 0.999999155031966
Mutual coherence 3: 0.9981787052765341


##### Sparse Optimization Problem

Given an overcomplete dictionary $A$ and a signal $y$, finding the best sparse representation of $y$ is equivalent to solving

$$
\min_x \|x\|_0 \quad \text{s.t.} \quad Ax=y
$$

Since this problem is NP-hard, we try to solve this problem instead:

**LASSO**

$$
\min_x \frac{1}{2} \| Ax-y \|_2^2 + \lambda \|x\|_1
$$

where

* A = wavelet dictionary
* y = signal
* x = sparse representation of the signal that we want to find
* $\lambda$ = regularization parameter

The wavelet dictionary is **highly overcomplete and coherent**, so greedy and first-order methods are unstable. Since the problem is convex and of moderate size, I used the global convex solver CVXPY to find the sparse signals.

In [ ]:
# Test signal sparsification
signal = 2 * wav_dict1[:,20] + 19 * wav_dict1[:,1900] - 4 * wav_dict1[:,3000]
sparsified_signal = wav.sparse_signal(signal,wav_dict1) # uses OMP
print(f'Sparsification error: {np.linalg.norm(wav_dict1 @ sparsified_signal - signal, ord=2)}')

Sparsification error: 7.99716456733055e-08


In [20]:
# Generate some signals

signal11 = 2 * wav_dict1[:,20] + 19 * wav_dict1[:,1900] - 4 * wav_dict1[:,3000]
signal12 = 2 * wav_dict2[:,20] + 19 * wav_dict2[:,1900] - 4 * wav_dict2[:,3000]
signal13 = 2 * wav_dict3[:,20] + 19 * wav_dict3[:,1900] - 4 * wav_dict3[:,3000]

signal21 = 23 * wav_dict1[:,1432] - 11 * wav_dict1[:,5555] + 17 * wav_dict1[:,7644]
signal22 = 23 * wav_dict2[:,1432] - 11 * wav_dict2[:,2378] + 17 * wav_dict2[:,3338]
signal23 = 23 * wav_dict3[:,1432] - 11 * wav_dict3[:,5555] + 17 * wav_dict3[:,7644]

random_signal = np.random.randn(L.shape[0])

In [15]:
# Find sparse signal representations
sparse_signal11 = wav.sparse_signal(signal11,wav_dict1)
sparse_signal12 = wav.sparse_signal(signal12,wav_dict2)
sparse_signal13 = wav.sparse_signal(signal13,wav_dict3)

sparse_signal21 = wav.sparse_signal(signal21,wav_dict1)
sparse_signal22 = wav.sparse_signal(signal22,wav_dict2)
sparse_signal23 = wav.sparse_signal(signal23,wav_dict3)

sparse_signal_random = wav.sparse_signal(random_signal,wav_dict3)

In [19]:
# Recovery Errors
err11 = np.linalg.norm(wav_dict1 @ sparse_signal11 - signal11, ord=2)
err12 = np.linalg.norm(wav_dict2 @ sparse_signal12 - signal12, ord=2)
err13 = np.linalg.norm(wav_dict3 @ sparse_signal13 - signal13, ord=2)

err21 = np.linalg.norm(wav_dict1 @ sparse_signal21 - signal21, ord=2)
err22 = np.linalg.norm(wav_dict2 @ sparse_signal22 - signal22, ord=2)
err23 = np.linalg.norm(wav_dict3 @ sparse_signal23 - signal23, ord=2)

err_random = np.linalg.norm(wav_dict3 @ sparse_signal_random - random_signal, ord=2)

print("Error signal 1 dictionary 1:", err11)
print("Error signal 1 dictionary 2:", err12)
print("Error signal 1 dictionary 3:", err13)

print("Error signal 2 dictionary 1:", err21)
print("Error signal 2 dictionary 2:", err22)
print("Error signal 2 dictionary 3:", err23)

print("Error random:", err_random)

Error signal 1 dictionary 1: 2.5621543119148667e-09
Error signal 1 dictionary 2: 1.5251579018258714e-09
Error signal 1 dictionary 3: 3.796555578905782e-08
Error signal 2 dictionary 1: 1.4237740259160458e-07
Error signal 2 dictionary 2: 1.3483902104331406e-08
Error signal 2 dictionary 3: 8.863916155603043e-09
Error random: 1.1105489781347665e-06
